# day 16

https://adventofcode.com/2021/day/16

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day16.txt')

LOGGER = logging.getLogger('day16')

## part 1

### problem statement:

#### loading data

In [ ]:
def hex_to_bin(h_str):
    l = len(h_str)
    return bin(int('0x' + h_str, 16))[2:].zfill(4 * l)

assert hex_to_bin('D2FE28') == '110100101111111000101000'

In [ ]:
test_data = []

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
def read_bin(b):
    """given a bin str, parse it (possibly hierarchical)"""
    ver = int(b[:3], 2)
    typ = int(b[3: 6], 2)
    
    if typ == 4:
        # literal value packet
        ptr = 6
        literal_parts = []
        while True:
            ind = b[ptr]
            vals = b[ptr + 1: ptr + 5]
            literal_parts.append(vals)
            ptr += 5
            if ind == '0':
                break
        contents = {'type': 'literal',
                    'value': int(''.join(literal_parts), 2)}
        remainder = b[ptr:]
    else:
        length_type_id = b[6]
        contents = {'type': 'operator'}
        
        if length_type_id == '0':
            # total bit length
            total_bit_length = int(b[7: 7 + 15], 2)
            contents['total_bit_length'] = total_bit_length
            
            # read the remaining string until we've consumed total_bit_length
            sub_packets = []
            consumed_so_far = 0
            remainder = b[7 + 15:]
            while consumed_so_far < total_bit_length:
                sub_packet, consumed, remainder = read_bin(remainder)
                sub_packets.append(sub_packet)
                consumed_so_far += consumed
            contents['value'] = sub_packets
        else:
            # num packets
            num_packets = int(b[7: 7 + 11], 2)
            contents['num_packets'] = num_packets
            
            # read the reamining string until we've built 3 sub packets
            sub_packets = []
            remainder = b[7 + 11:]
            for i in range(num_packets):
                sub_packet, consumed, remainder = read_bin(remainder)
                sub_packets.append(sub_packet)
            contents['value'] = sub_packets
    consumed = len(b) - len(remainder)
    return {'version': ver, 'type': typ, 'contents': contents}, consumed, remainder

assert read_bin(hex_to_bin('D2FE28')) == (
    {'version': 6,
     'type': 4,
     'contents': {'type': 'literal', 'value': 2021}},
    21,
    '000'
)
assert read_bin(hex_to_bin('38006F45291200')) == (
    {'version': 1,
     'type': 6,
     'contents': {'type': 'operator',
                  'total_bit_length': 27,
                  'value': [{'version': 6, 'type': 4, 'contents': {'type': 'literal', 'value': 10}},
                            {'version': 2, 'type': 4, 'contents': {'type': 'literal', 'value': 20}}]}},
    49,
    '0000000'
)
assert read_bin(hex_to_bin('EE00D40C823060')) ==(
    {'version': 7,
     'type': 3,
     'contents': {'type': 'operator',
                  'num_packets': 3,
                  'value': [{'version': 2, 'type': 4, 'contents': {'type': 'literal', 'value': 1}},
                            {'version': 4, 'type': 4, 'contents': {'type': 'literal', 'value': 2}},
                            {'version': 1, 'type': 4, 'contents': {'type': 'literal', 'value': 3}}]}},
    51,
    '00000'
)

In [ ]:
def get_versions(d, versions=None):
    if versions is None:
        versions = []
    versions.append(d['version'])
    if d['contents']['type'] == 'operator':
        for sub_packet in d['contents']['value']:
            versions = get_versions(sub_packet, versions)
    return versions

assert get_versions(read_bin(hex_to_bin('D2FE28'))[0]) == [6]
assert get_versions(read_bin(hex_to_bin('38006F45291200'))[0]) == [1, 6, 2]
assert get_versions(read_bin(hex_to_bin('EE00D40C823060'))[0]) == [7, 2, 4, 1]

#### function def

In [ ]:
def q_1(data):
    return sum(get_versions(read_bin(hex_to_bin(data))[0]))

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1('8A004A801A8002F478') == 16
    assert q_1('620080001611562C8802118E34') == 12
    assert q_1('C0015000016115A2E0802F182340') == 23
    assert q_1('A0016C880162017C3686B18A3D4780') == 31
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def get_value(d):
    dt = d['type']
    values = d['contents']['value']
    if dt == 0:
        return sum([get_value(_) for _ in values])
    elif dt == 1:
        p = 1
        for sub_packet in values:
            p *= get_value(sub_packet)
        return p
    elif dt == 2:
        return min([get_value(_) for _ in values])
    elif dt == 3:
        return max([get_value(_) for _ in values])
    elif dt == 4:
        return values
    elif dt == 5:
        a, b = values
        return int(get_value(a) > get_value(b))
    elif dt == 6:
        a, b = values
        return int(get_value(a) < get_value(b))
    elif dt == 7:
        a, b = values
        return int(get_value(a) == get_value(b))
    print(dt)
    print(d)
    raise NotImplementedError()

In [ ]:
def q_2(data):
    d = read_bin(hex_to_bin(data))[0]
    return get_value(d)

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2('C200B40A82') == 3
    assert q_2('04005AC33890') == 54
    assert q_2('880086C3E88112') == 7
    assert q_2('CE00C43D881120') == 9
    assert q_2('D8005AC2A8F0') == 1
    assert q_2('F600BC2D8F') == 0
    assert q_2('9C005AC2F8F0') == 0
    assert q_2('9C0141080250320F1802104A08') == 1
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin